In [69]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('rheeza_pharm').getOrCreate()

## Understand the dataset

In [70]:
trial_df = spark.read.json('dataset.json', multiLine= True)
trial_df.show(3)

+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|ageofparticipant|           clinician|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|              result|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|              19|{Ontario, Saul, t...|  Placebo|    1619827200000|      1617235200000|                            52|{BP normalized, r...|
|              14|{Ontario, Saul, n...| Naproxen|    1619827200000|      1617235200000|                            78|    {Follow-up, N/A}|
|              17|{Ontario, Saul, n...|  Placebo|    1619827200000|      1617235200000|                            14|    {Follow-up, N/A}|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
only showing top 3 r

In [71]:
trial_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- clinician: struct (nullable = true)
 |    |-- branch: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- conclusion: string (nullable = true)
 |    |-- sideeffectsonparticipant: string (nullable = true)



In [72]:
trial_df.dtypes

[('ageofparticipant', 'bigint'),
 ('clinician', 'struct<branch:string,name:string,role:string>'),
 ('drug_used', 'string'),
 ('experimentenddate', 'string'),
 ('experimentstartdate', 'string'),
 ('noofhourspassedatfirstreaction', 'bigint'),
 ('result', 'struct<conclusion:string,sideeffectsonparticipant:string>')]

In [73]:
trial_df.columns

['ageofparticipant',
 'clinician',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result']

In [74]:
columns= ['ageofparticipant',
 'clinician.branch',
 'clinician.name',
 'clinician.role',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result.conclusion',
 'result.sideeffectsonparticipant']

In [75]:
# flattening the nested column
trial_df.select(columns).show(5) 

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|
|              17|Ontario|   Saul|    nurse|  Placebo|    1619827200000|      1617235200000|                            14|    Follow-up|                     N/A|
|              18|Onta

In [76]:
from pyspark.sql import functions as fn

In [77]:
#  displaying count of null values in the columns
trial_df.select([fn.count(fn.when(fn.col(column).isNull(), column)).alias(column) for column in columns]).show()

+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|ageofparticipant|clinician.branch|clinician.name|clinician.role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|result.conclusion|result.sideeffectsonparticipant|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|               0|               0|             0|           109|        0|                0|                  0|                            73|               53|                              0|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+



## Cleaning

In [78]:
# flattening the data
new_trials_df = trial_df.select(columns)
new_trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- branch: string (nullable = true)
 |-- name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- conclusion: string (nullable = true)
 |-- sideeffectsonparticipant: string (nullable = true)



In [79]:
#  Renaming columns
new_column_name = {
    'ageofparticipant': 'age_of_participant'
    ,'branch': 'clinic_branch'
    ,'name': 'head_clinician'
    ,'role': 'assistants_role'
    ,'experimentenddate': 'experiment_end_date'
    ,'experimentstartdate': 'experiment_start_date'
    ,'experimentenddate': 'experiment_end_date'
    ,'noofhourspassedatfirstreaction': 'hours_passed_at_first_reaction'
    ,'sideeffectsonparticipant': 'observed_side_effects'
    }

new_trials_df= new_trials_df.withColumnsRenamed(new_column_name)

In [80]:
new_trials_df.show(2)

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+---------------------+
|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effects|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+---------------------+
|                19|      Ontario|          Saul|      therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|       rashes on neck|
|                14|      Ontario|          Saul|          nurse| Naproxen|      1619827200000|        1617235200000|                            78|    Follow-up|                  N/A|
+------------------+-------------+--------------+---------------+---------+

In [81]:
new_trials_df.printSchema()

root
 |-- age_of_participant: long (nullable = true)
 |-- clinic_branch: string (nullable = true)
 |-- head_clinician: string (nullable = true)
 |-- assistants_role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experiment_end_date: string (nullable = true)
 |-- experiment_start_date: string (nullable = true)
 |-- hours_passed_at_first_reaction: long (nullable = true)
 |-- conclusion: string (nullable = true)
 |-- observed_side_effects: string (nullable = true)



In [82]:
new_trials_df.dtypes

[('age_of_participant', 'bigint'),
 ('clinic_branch', 'string'),
 ('head_clinician', 'string'),
 ('assistants_role', 'string'),
 ('drug_used', 'string'),
 ('experiment_end_date', 'string'),
 ('experiment_start_date', 'string'),
 ('hours_passed_at_first_reaction', 'bigint'),
 ('conclusion', 'string'),
 ('observed_side_effects', 'string')]

In [83]:
new_trials_df.describe().show()

+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------+-------------+---------------------+
|summary|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used| experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effects|
+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------+-------------+---------------------+
|  count|              3586|         3586|          3586|           3477|     3586|                3586|                 3586|                          3513|         3533|                 3586|
|   mean|17.507250418293363|         null|          null|           null|     null|1.618381578137200...| 1.615813671834913...|             44.89097637346997|         null|                 null|
| stddev|2.3066401927555233|  

In [84]:
#  filling null values
new_trials_df = new_trials_df.na.fill({'conclusion': 'unknown', 'assistants_role': 'unknown', 'hours_passed_at_first_reaction': 0})

In [85]:
new_trials_df.describe().show()

+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------+-------------+---------------------+
|summary|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used| experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effects|
+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------+-------------+---------------------+
|  count|              3586|         3586|          3586|           3586|     3586|                3586|                 3586|                          3586|         3586|                 3586|
|   mean|17.507250418293363|         null|          null|           null|     null|1.618381578137200...| 1.615813671834913...|              43.9771332961517|         null|                 null|
| stddev|2.3066401927555233|  

## Transformation

In [86]:
new_trials_df.printSchema()

root
 |-- age_of_participant: long (nullable = true)
 |-- clinic_branch: string (nullable = true)
 |-- head_clinician: string (nullable = true)
 |-- assistants_role: string (nullable = false)
 |-- drug_used: string (nullable = true)
 |-- experiment_end_date: string (nullable = true)
 |-- experiment_start_date: string (nullable = true)
 |-- hours_passed_at_first_reaction: long (nullable = false)
 |-- conclusion: string (nullable = false)
 |-- observed_side_effects: string (nullable = true)



In [87]:
from pyspark.sql import types as dtypes

In [88]:
# convert datatype
new_trials_df.withColumn('end ts', fn.col('experiment_end_date').cast(dtypes.LongType())).show(3)

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+---------------------+-------------+
|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effects|       end ts|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+---------------------+-------------+
|                19|      Ontario|          Saul|      therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|       rashes on neck|1619827200000|
|                14|      Ontario|          Saul|          nurse| Naproxen|      1619827200000|        1617235200000|                            78|    Follow-up|                  N/A|1619827200000|
|    

In [89]:
# convert data types to integers (long)
# divide by 1000
# convert from unix to datetime

new_trials_df = new_trials_df\
    .withColumn('start ts', fn.from_unixtime(fn.col('experiment_start_date').cast(dtypes.LongType())/1000, "yyyy-MM-dd HH:mm:ss")) \
    .withColumn('start ts', fn.col('start ts').cast(dtypes.TimestampType()))\
        .withColumn('end ts', fn.from_unixtime(fn.col('experiment_end_date').cast(dtypes.LongType())/1000, "yyyy-MM-dd HH:mm:ss")) \
        .withColumn('end ts', fn.col('end ts').cast(dtypes.TimestampType()))

In [90]:
new_trials_df.dtypes

[('age_of_participant', 'bigint'),
 ('clinic_branch', 'string'),
 ('head_clinician', 'string'),
 ('assistants_role', 'string'),
 ('drug_used', 'string'),
 ('experiment_end_date', 'string'),
 ('experiment_start_date', 'string'),
 ('hours_passed_at_first_reaction', 'bigint'),
 ('conclusion', 'string'),
 ('observed_side_effects', 'string'),
 ('start ts', 'timestamp'),
 ('end ts', 'timestamp')]

In [91]:
new_trials_df.show(3)

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+---------------------+-------------------+-------------------+
|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effects|           start ts|             end ts|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+---------------------+-------------------+-------------------+
|                19|      Ontario|          Saul|      therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|       rashes on neck|2021-04-01 01:00:00|2021-05-01 01:00:00|
|                14|      Ontario|          Saul|          nurse| Naproxen|      1619827200000|     